<a href="https://colab.research.google.com/github/MehdiParviz/Atrial-fibrillation-risk-prediction/blob/main/Atrial_fibrillation_risk_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description

Scores | Italian score	| Mayo score	| Treatment-aware | Treatment-aware with interaction*
--- |--- | --- | --- | ---
Aim	| Prevalence and risk factors of AF in a cohort of ibrutinib-naive CLL| Prevalence of AF in a general cohort of CLL | Prevalence and risk factors of AF in a cohort of CLL | Prevalence and risk factors of AF in a cohort of CLL
 -| Validated on a cohort of ibrutinib-treated patients | - | Ibrutinib and other treatment types included | Ibrutinib and other treatment types included
Prediction point | Diagnosis | Diagnosis	| Treatment initiation | Treatment initiation
Risk factors | | | |
Age ≥ 65 | 1 | 2 | 1 | 1 (2)
Age ≥ 75 |   | 1 | 1 | 1 (2)
Male sex | 1 | 1 | 1 | 1 (2)
Hypertension |  | 1 | 1 | 1 (2)
Cardiopathy | 3 |  | |
Valvular heart diseases | 2 | 2 | 1 | 1
Hypo/hyperthyroidism | 2 |  |  |
Chronic lung diseases | 1 |  | |
Type-2 diabetes | 1 |  | 1 | 1 (2)
Grade ≥ 3 infections | 1 |  | |
Blood culture (count) | | | 1 | 1 (2)
Blood culture (positive) | | | 1 | 1 (2)
Ibrutinib treatment | | | 1 | 1
* **The score in the parenthesis applies if treated with ibrutinib**

## Implmentation

In [40]:
import ipywidgets as widgets
from IPython.display import display, clear_output


edit_age = widgets.Text(description="Age:", value='65', style={'description_width': '150px'})
edit_bcc = widgets.Text(description="Blood culture (count):", value='0', style={'description_width': '150px'})
edit_italian = widgets.Text(description="Italian score:", layout=widgets.Layout(width='300px'), style={'description_width': '230px'})
edit_mayo = widgets.Text(description="Mayo score:", layout=widgets.Layout(width='300px'), style={'description_width': '230px'})
edit_ta = widgets.Text(description="Treatment-aware:", layout=widgets.Layout(width='300px'), style={'description_width': '230px'})
edit_tai = widgets.Text(description="Treatment-aware with interaction:", layout=widgets.Layout(width='300px'), style={'description_width': '230px'})

# Define the dropdown
dd_sex = widgets.Dropdown(
    options=['Male', 'Female'],
    description='Sex:', style={'description_width': '150px'}
)

chbx_ht      = widgets.Checkbox(value=False, description='Hypertension', disabled=False)
chbx_vhd     = widgets.Checkbox(value=False, description='Valvular heart diseases', disabled=False)
chbx_crd     = widgets.Checkbox(value=False, description='Cardiopathy', disabled=False)
chbx_hpr_thr = widgets.Checkbox(value=False, description='Hypothyroidism', disabled=False)
chbx_hpo_thr = widgets.Checkbox(value=False, description='Hyperthyroidism', disabled=False)
chbx_cld     = widgets.Checkbox(value=False, description='Chronic lung diseases', disabled=False)
chbx_t2      = widgets.Checkbox(value=False, description='Type-2 diabetes', disabled=False)
chbx_inf     = widgets.Checkbox(value=False, description='Grade ≥ 3 infections', disabled=False)
chbx_bcp     = widgets.Checkbox(value=False, description='Blood culture (positive)', disabled=False)
chbx_ibr     = widgets.Checkbox(value=False, description='Ibrutinib treatment', disabled=False)

compute_button = widgets.Button(description='Compute scores',
                                 disabled=False,
                                 button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                 tooltip='Click me',
                                 icon='check',
                                 layout={'width': '150px'}
)

def comp_scores(b):
    out.clear_output()
    age = edit_age.value
    #assert age.isdigit(), "Input is not an integer"
    if not age.replace('.','').replace('-','').isnumeric():
      with out:
        print('Entered age is not an integer')
      return 0

    age = float(age)
    if age<=0:
      with out:
        print("Entered age is not positive")
      return 0

    bc_count = edit_bcc.value
    #assert bc_count.isdigit(), "Input is not an integer"
    if not bc_count.replace('-','').isnumeric():
      with out:
        print("Entered blood culture count is not an integer")
      return 0
    bc_count = int(bc_count)
    if bc_count<0:
      with out:
        print("Entered blood culture count is negative")
      return 0


    is_male = dd_sex.value == 'Male'

    has_ht = chbx_ht.value
    has_vhd = chbx_vhd.value
    has_crd = chbx_crd.value
    has_hpr_thr = chbx_hpr_thr.value
    has_hpo_thr = chbx_hpo_thr.value
    has_cld = chbx_cld.value
    has_t2 = chbx_t2.value
    has_inf = chbx_inf.value
    has_bcp = chbx_bcp.value
    has_ibr = chbx_ibr.value

    italian_score = (age > 65) + is_male + 3 * has_crd + 2 * has_vhd + 2 * (has_hpo_thr | has_hpr_thr) + has_cld + has_t2 + has_inf
    mayo_score = 2 * (age >= 65) + (age >= 75) + is_male + 2 * has_vhd + has_ht
    ta_score = has_ibr + (age > 65) + (age >= 75) + is_male + has_vhd + has_ht + has_t2 + has_bcp + (bc_count > 5)
    tai_score = has_ibr + (1 + has_ibr) * ((age > 65) + (age >= 75) + is_male + has_ht + has_t2 + has_bcp + (bc_count > 5)) + has_vhd

    # Output message
    edit_italian.value = str(italian_score)
    edit_mayo.value = str(mayo_score)
    edit_ta.value = str(ta_score)
    edit_tai.value = str(tai_score)

out = widgets.Output()#layout={'border': '1px solid black'})
compute_button.on_click(comp_scores)
numeric_boxes = widgets.VBox([edit_age, edit_bcc, dd_sex])
chbx_boxes = widgets.VBox([chbx_ibr, chbx_ht, chbx_vhd, chbx_crd, chbx_hpr_thr, chbx_hpo_thr, chbx_cld, chbx_t2, chbx_inf, chbx_bcp])
score_boxes = widgets.VBox([edit_italian, edit_mayo, edit_ta, edit_tai])
all_boxes = widgets.HBox([numeric_boxes, chbx_boxes, compute_button, score_boxes])
display(all_boxes, out)

Output()